In [1]:
import pandas as pd
import numpy as np
import requests
import os 
from dotenv import load_dotenv
import requests
from collections import Counter
from fuzzywuzzy import process, fuzz
from tqdm import tqdm
import pickle
import re
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_columns = None
pd.set_option('display.float_format', lambda x: '%.3f' % x)

import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append("../")

import src.biblioteca as bb

Note to self:
en este archivo van cosas que luego van a ser tres archivos distintos.
- Limpieza
- EDA
- Charts
- 
Recomendable separarlo y trabajarlo de forma independiente

# Input files

Base data file

In [2]:
with open('../data/pickle/dataset_usa_base.pickle', 'rb') as base_data:
    base_data = pickle.load(base_data)
print(base_data.shape)
base_data.head(1)

(455057, 9)


,title,rank,date,artist,url,region,chart,trend,streams
8246,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.000


LastFM extraction

In [3]:
with open('../data/pickle/spotify_dataset_lastfm.pickle', 'rb') as data_lastfm:
    data_lastfm = pickle.load(data_lastfm)
data_lastfm.drop_duplicates(inplace=True)
print(data_lastfm.drop_duplicates().shape)
data_lastfm.head(1)

(14659, 17)


,url,artist,track,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4
8246,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert),"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap


Spotify extraction

In [4]:
with open('../data/pickle/spotify_dataset_audioFeatures.pickle', 'rb') as data_spotify:
    data_spotify = pickle.load(data_spotify)
data_spotify.drop_duplicates(inplace=True)
print(data_spotify.drop_duplicates().shape)
data_spotify.head(1)

(13989, 21)


,url,artist,track,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
0,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert),spotify:track:0M9ydKzuF3oZTfYYPfaGX1,0.926,0.666,11.000,-5.314,1.000,0.244,0.061,0.000,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.000,4.000


# Merging all data

In [5]:
df = base_data.merge(data_lastfm, on = 'url', indicator = True, how = 'left')
df.value_counts('_merge')

_merge
both          456306
left_only          0
right_only         0
dtype: int64

In [6]:
df.drop(['_merge'], axis = 1, inplace = True)
df = df.merge(data_spotify, on = 'url', indicator = True, how = 'left')
df.value_counts('_merge')

_merge
both          442906
left_only      13924
right_only         0
dtype: int64

In [7]:
df.drop(['artist_y', 'track_x', 'artist', 'track_y', '_merge'], axis = 1, inplace= True)
df.rename({'artist_x':'artist'}, inplace=True, axis=1)
print(df.shape)
df.head(1)

(456830, 41)


,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
0,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.000,"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap,spotify:track:0M9ydKzuF3oZTfYYPfaGX1,0.926,0.666,11.000,-5.314,1.000,0.244,0.061,0.000,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.000,4.000


# Cleaning data

In [8]:
df.head(1)

,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
0,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.000,"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap,spotify:track:0M9ydKzuF3oZTfYYPfaGX1,0.926,0.666,11.000,-5.314,1.000,0.244,0.061,0.000,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.000,4.000


Highest rank in list + Weeks in charts

In [9]:
lista_url = df['url'].unique().tolist()
len(lista_url)

14650

In [10]:
ranks = {
    'url':[],
    'highest_rank':[],
    'weeks_in_charts':[]
}

In [11]:
for url in tqdm(lista_url):
    ranks['url'].append(url)
    ranks['highest_rank'].append(df[df['url'] == url].sort_values(by = 'rank').iloc[0,1])
    ranks['weeks_in_charts'].append(df[df['url'] == url].shape[0])

100%|██████████| 14650/14650 [14:15<00:00, 17.13it/s]


In [12]:
print(len(ranks['url']), len(ranks['highest_rank']), len(ranks['weeks_in_charts']))

14650 14650 14650


In [13]:
df = df.merge(pd.DataFrame(ranks), on = 'url', indicator= True)
df.value_counts('_merge')

_merge
both          456830
left_only          0
right_only         0
dtype: int64

In [14]:
df.drop(['_merge'], axis = 1, inplace=True)

Main music genre

Primo los géneros de las canciones -- si no hubiera datos, cojo los valores del artista

In [15]:
df.head(1)

,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,highest_rank,weeks_in_charts
0,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.000,"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap,spotify:track:0M9ydKzuF3oZTfYYPfaGX1,0.926,0.666,11.000,-5.314,1.000,0.244,0.061,0.000,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.000,4.000,1,429


In [16]:
df['music_genre'] = np.where(df['aritist_genre_0'].isnull() == True, df['track_genre_0'].str.lower(), df['aritist_genre_0'].str.lower())
df['music_genre'].value_counts()

hip-hop         131641
pop              65166
rap              30438
country          25113
rnb              22598
                 ...  
wave                 1
calypso              1
southern rap         1
salsa                1
c                    1
Name: music_genre, Length: 363, dtype: int64

In [17]:
cuenta_generos = Counter(genres for genres in df['music_genre'])
print(f"Hay {len(cuenta_generos)} generos diferentes.")

Hay 2976 generos diferentes.


In [18]:
dict_genres = dict(cuenta_generos.most_common(24))

In [19]:
fuzz.ratio('hip-hop', "hip hop")

86

In [20]:
def music_genres(column, genres):
    max = 0
    for key in genres.keys():
        try:
            similar = fuzz.ratio(column, key)
            if similar > max:
                max = similar
                genre = key     
        except:
            return "other"
    if max > 60:
        return genre
    else:
        return "other"

In [21]:
df["clean_music_genre"] = df.apply(lambda x: music_genres(x["music_genre"], dict_genres), axis = 1)
df["clean_music_genre"].value_counts()

hip-hop             132882
other                91631
pop                  65166
rap                  30446
country              25295
rnb                  22649
trap                 19979
electronic           11784
indie                11121
rock                  6456
soul                  5459
acoustic              3902
indie pop             3526
classic rock          3410
jazz                  3110
k-pop                 2783
house                 2610
lo-fi                 2416
indie rock            2098
latin                 2065
female vocalists      1823
black metal           1772
cloud rap             1600
reggaeton             1443
seen live             1404
Name: clean_music_genre, dtype: int64

Gender from artists

ojo, hay un poco de error -- volver a pasarlo, pero solo con el nombre de los artistas (que no sean grupos)

In [22]:
df.sample(5)

,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,highest_rank,weeks_in_charts,music_genre,clean_music_genre
212012,Believer,198,2019-03-26,Imagine Dragons,https://open.spotify.com/track/0pqnGHJpmpxLKif...,United States,top200,NEW_ENTRY,212460.000,"02 Feb 2017, 03:09","07 Apr 2009, 22:33",Imagine Dragons is a Grammy award-winning alte...,Imagine Dragons is a Grammy award-winning alte...,indie,indie rock,alternative,rock,indie pop,rock,pop,alternative,pop rock,IMAGINE DRAGONS,spotify:track:0pqnGHJpmpxLKifKRmU6WP,0.776,0.780,10.000,-4.374,0.000,0.128,0.062,0.000,0.081,0.666,124.949,audio_features,0pqnGHJpmpxLKifKRmU6WP,https://api.spotify.com/v1/tracks/0pqnGHJpmpxL...,https://api.spotify.com/v1/audio-analysis/0pqn...,204347.000,4.000,121,206,indie,indie
126156,Whiskey Glasses,148,2019-11-18,Morgan Wallen,https://open.spotify.com/track/6foY66mWZN0pSRj...,United States,top200,MOVE_UP,234561.000,"06 Dec 2021, 23:10","14 Jan 2018, 02:26",Morgan Wallen is a country music singer and so...,Morgan Wallen is a country music singer and so...,country,racist,male vocalist,the voice,racism,NaN,NaN,NaN,NaN,NaN,spotify:track:6foY66mWZN0pSRjZ408c00,0.614,0.680,6.000,-4.580,1.000,0.029,0.369,0.000,0.115,0.707,149.959,audio_features,6foY66mWZN0pSRjZ408c00,https://api.spotify.com/v1/tracks/6foY66mWZN0p...,https://api.spotify.com/v1/audio-analysis/6foY...,234347.000,4.000,53,971,country,country
196481,Stay (with Alessia Cara),13,2017-05-23,Zedd,https://open.spotify.com/track/0dA2Mk56wEzDgeg...,United States,top200,MOVE_DOWN,891167.000,"25 Feb 2017, 17:04","16 May 2011, 05:54",\nZedd is a Grammy-award winning electro-house...,\nZedd is a Grammy-award winning electro-house...,electronic,electro house,House,electro,dubstep,alessia cara,electropop,stay,zedd,future bass,spotify:track:0vhyWEEpOwmBtYWm0D44oN,0.689,0.582,5.000,-5.785,0.000,0.068,0.271,0.000,0.112,0.555,102.050,audio_features,0vhyWEEpOwmBtYWm0D44oN,https://api.spotify.com/v1/tracks/0vhyWEEpOwmB...,https://api.spotify.com/v1/audio-analysis/0vhy...,210880.000,4.000,3,296,electronic,electronic
109607,We Paid (feat. 42 Dugg),151,2021-01-04,Lil Baby,https://open.spotify.com/track/6gxKUmycQX7uyMw...,United States,top200,MOVE_DOWN,227429.000,"11 May 2021, 22:47","20 Jul 2018, 18:28",Rapper Lil Baby grew up in Atlanta hanging wit...,Rapper Lil Baby grew up in Atlanta hanging wit...,Hip-Hop,rap,hip hop,trap,atlanta,rap,hiphop,trap,atlanta,lil baby,spotify:track:6gxKUmycQX7uyMwJcweFjp,0.926,0.542,1.000,-5.064,1.000,0.293,0.135,0.000,0.088,0.564,135.140,audio_features,6gxKUmycQX7uyMwJcweFjp,https://api.spotify.com/v1/tracks/6gxKUmycQX7u...,https://api.spotify.com/v1/audio-analysis/6gxK...,181929.000,4.000,9,311,hip-hop,hip-hop
195907,Be the One,193,2017-02-23,Dua Lipa,https://open.spotify.com/track/7FCfMXYTIiQ9b4h...,United States,top200,NEW_ENTRY,168131.000,"12 Mar 2017, 14:01","26 Apr 2016, 03:42","Dua Lipa (born August 22, 1995) is an English ...","Dua Lipa (born August 22, 1995) is an English ...",pop,electropop,synthpop,british,female vocalists,pop,synthpop,dance,british,emo,spotify:track:5GwtWeTDyQ6tr1NdtpAsu7,0.661,0.651,7.000,-3.771,0.000,0.050,0.117,0.000,0.056,0.368,87.460,audio_features,5GwtWeTDyQ6tr1NdtpAsu7,https://api.spotify.com/v1/tracks/5GwtWeTDyQ6t...,https://api.spotify.com/v1/audio-analysis/5Gwt...,202915.000,4.000,165,18,pop,pop


In [25]:
## funciona pero tiene un poco de error
def generos(col):
    for key, value in bb.dict_gender.items():
        for k in key:
            for item in value:
                try:
                    if item in col.lower():
                        return key
                except:
                    return np.nan
    return np.nan

In [29]:
def generos_2(col):
    for key, value in bb.dict_gender.items():
        try:
            if value in col.lower():
                return key
        except:
            return np.nan
    return np.nan

In [43]:
df.head(1)

,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,highest_rank,weeks_in_charts,music_genre,clean_music_genre,gender
0,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.000,"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap,spotify:track:0M9ydKzuF3oZTfYYPfaGX1,0.926,0.666,11.000,-5.314,1.000,0.244,0.061,0.000,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.000,4.000,1,429,hip-hop,hip-hop,NaN


In [53]:
#df["gender"] = df.apply(lambda x: generos(x['content'], bb.dict_gender), axis=1)
df["gender"] = df['summary'].apply(generos)

In [54]:
df['gender'].value_counts() / df.shape[0]

male         0.389
female       0.136
group        0.136
non-binary   0.022
Name: gender, dtype: float64

Age from artist

-- taking the data from the summary

In [72]:
df.sample(5)

,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,highest_rank,weeks_in_charts,music_genre,clean_music_genre,gender
251141,El Manual,175,2020-07-07,Anuel AA,https://open.spotify.com/track/7kOf0qHO4Rcwk6Q...,United States,top200,NEW_ENTRY,223299.000,NaN,"27 Oct 2018, 16:30","Emmanuel Gazmey Santiago, known professionally...","Emmanuel Gazmey Santiago, known professionally...",Reggaeton,trap,Hip-Hop,latin,rap,NaN,NaN,NaN,NaN,NaN,spotify:track:7kOf0qHO4Rcwk6QJifZDRJ,0.843,0.845,0.000,-2.344,1.000,0.050,0.350,0.002,0.053,0.898,126.989,audio_features,7kOf0qHO4Rcwk6QJifZDRJ,https://api.spotify.com/v1/tracks/7kOf0qHO4Rcw...,https://api.spotify.com/v1/audio-analysis/7kOf...,210624.000,4.000,70,5,reggaeton,reggaeton,NaN
249666,In My Head,18,2018-12-08,Peter Manos,https://open.spotify.com/track/1tT55K6VEyO6XFD...,United States,viral50,MOVE_DOWN,NaN,NaN,"01 Jan 1970, 00:00","<a href=""https://www.last.fm/music/Peter+Mano...",,indie,folk,acoustic,pop,USA,NaN,NaN,NaN,NaN,NaN,spotify:track:3bne7Qit5AbHkX6kWDItYP,0.499,0.181,9.000,-15.359,0.000,0.043,0.745,0.001,0.106,0.075,111.956,audio_features,3bne7Qit5AbHkX6kWDItYP,https://api.spotify.com/v1/tracks/3bne7Qit5AbH...,https://api.spotify.com/v1/audio-analysis/3bne...,214533.000,4.000,10,73,indie,indie,NaN
410926,Mayonaka no Door / Stay With Me,1,2020-12-10,Miki Matsubara,https://open.spotify.com/track/2BHj31ufdEqVK5C...,United States,viral50,SAME_POSITION,NaN,NaN,"08 Aug 2010, 11:12","Miki Matsubara (松原 みき, Matsubara Miki) (Novemb...","Miki Matsubara (松原 みき, Matsubara Miki) (Novemb...",j-pop,japanese,pop,JPop,gundam,NaN,NaN,NaN,NaN,NaN,spotify:track:2BHj31ufdEqVK5CkYDp9mA,0.658,0.747,5.000,-6.615,1.000,0.043,0.176,0.000,0.079,0.792,108.259,audio_features,2BHj31ufdEqVK5CkYDp9mA,https://api.spotify.com/v1/tracks/2BHj31ufdEqV...,https://api.spotify.com/v1/audio-analysis/2BHj...,312293.000,4.000,1,68,j-pop,k-pop,group
172666,Forbidden Lovers,3,2017-08-27,King Garbage,https://open.spotify.com/track/2y89a0zvMiELpbS...,United States,viral50,NEW_ENTRY,NaN,NaN,"01 Jan 1970, 00:00","<a href=""https://www.last.fm/music/king+garba...",,experimental,electronic,trip-hop,soul,USA,NaN,NaN,NaN,NaN,NaN,spotify:track:4H9zRISm23B4XL6TM5xBO7,0.773,0.633,11.000,-8.783,0.000,0.090,0.415,0.001,0.252,0.831,90.030,audio_features,4H9zRISm23B4XL6TM5xBO7,https://api.spotify.com/v1/tracks/4H9zRISm23B4...,https://api.spotify.com/v1/audio-analysis/4H9z...,170807.000,4.000,3,3,experimental,other,NaN
215524,Lucky You (feat. Joyner Lucas),5,2018-09-21,Eminem,https://open.spotify.com/track/60SdxE8apGAxMiR...,United States,top200,SAME_POSITION,1346485.000,"28 May 2019, 23:56","11 Feb 2006, 00:14","Marshall Bruce Mathers III (born October 17, 1...","Marshall Bruce Mathers III (born October 17, 1...",rap,Hip-Hop,Eminem,hip hop,pop,Hip-Hop,rap,joyner lucas,hip hop,2018,spotify:track:60SdxE8apGAxMiRrpbmLY0,0.876,0.786,10.000,-4.884,0.000,0.306,0.063,0.000,0.127,0.575,153.068,audio_features,60SdxE8apGAxMiRrpbmLY0,https://api.spotify.com/v1/tracks/60SdxE8apGAx...,https://api.spotify.com/v1/audio-analysis/60Sd...,244679.000,4.000,1,217,rap,rap,male


In [114]:
df['birthday'] =  df['content'].apply(lambda x: re.findall(r'\w* \d*, \d{4}',str(x))).str[0]
df['birthday'].value_counts()

December 2, 1998     10839
July 4, 1995         10700
January 23, 1998      9272
October 24, 1986      8719
December 18, 2001     7645
                     ...  
April 14, 1993           1
May 6, 2008              1
 1970, 1971              1
August 19, 1986          1
August 30, 1997          1
Name: birthday, Length: 999, dtype: int64

In [120]:
df.sample()

,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,highest_rank,weeks_in_charts,music_genre,clean_music_genre,gender,birthday
146998,Hate Me (with Juice WRLD),162,2020-06-16,Ellie Goulding,https://open.spotify.com/track/6kls8cSlUyHW2BU...,United States,top200,SAME_POSITION,220758.000,"08 Dec 2019, 22:55","12 May 2008, 15:32","Ellie Goulding (born Elena Jane Goulding, 30 D...","Ellie Goulding (born Elena Jane Goulding, 30 D...",female vocalists,electronic,british,indie,indie pop,2010s,2019,pop,ellie goulding,Juice WRLD,spotify:track:6kls8cSlUyHW2BUOkDJIZE,0.657,0.768,8.000,-4.628,1.000,0.051,0.108,0.000,0.144,0.759,75.025,audio_features,6kls8cSlUyHW2BUOkDJIZE,https://api.spotify.com/v1/tracks/6kls8cSlUyHW...,https://api.spotify.com/v1/audio-analysis/6kls...,186223.000,4.000,13,382,female vocalists,female vocalists,female,NaN


# EDA

# Questions / hypothesis

Most streamed artists

In [ ]:
top10 = df.groupby(['artist'])['streams'].sum().reset_index().sort_values(by = 'streams', ascending = False).head(10)
top10['ratio'] = top10['streams'] * 100 / (df[~df['streams'].isnull()]['streams'].sum())
top10

Most streamed music genres

In [ ]:
top10genre = df.groupby(['aritist_genre_0'])['streams'].sum().reset_index().sort_values(by = 'streams', ascending = False).head(10)
top10genre['ratio'] = top10genre['streams'] * 100 / (df[~df['streams'].isnull()]['streams'].sum())
top10genre[top10genre['ratio'] > 0]

Artist evolution

In [ ]:
lista_artistas = list(top10['artist'])
df_top10 = df[df['artist'].isin(lista_artistas)]
df_top10.head()

In [ ]:
num_rows = 20
years = list(range(1990, 1990 + num_rows))
data_preproc = pd.DataFrame({
    'Year': years, 
    'A': np.random.randn(num_rows).cumsum(),
    'B': np.random.randn(num_rows).cumsum(),
    'C': np.random.randn(num_rows).cumsum(),
    'D': np.random.randn(num_rows).cumsum()})

In [ ]:
sns.lineplot(x='Year', y='value', hue='variable', 
             data=pd.melt(data_preproc, ['Year']))

In [ ]:
test = pd.pivot_table(df_top10[df_top10['chart'] == 'top200'][['date', 'artist', 'rank']], index = 'date', values = ['rank'], columns = 'artist').reset_index().droplevel(level=0, axis =1)
test.rename({'':'date'}, axis = 1, inplace=True)
print(test.shape)
test.head()

In [ ]:
sns.lineplot(x='Year', y='value', hue='variable', 
             data=pd.melt(data_preproc, ['Year']))

In [ ]:
sns.set(rc={'figure.figsize':(50,8.27)})

c = sns.lineplot(data = test, x = test['date'].unique().tolist(), y = 'Ariana Grande')
c.set_xticklabels(labels = test['date'].unique().tolist(), rotation=90)

Music genre evolution

In [ ]:
df